In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from tqdm import tqdm
import pandas as pd

In [2]:
PATH = r'C:\Users\ADMIN\Desktop\New folder\cleaned_data.csv'
df = pd.read_csv(PATH)

In [3]:
df

,Label,Cleaned_Name
0,ghế văn phòng,ghế
1,thước dây,thước dây sợi thủy tinh
2,cưa tay,cưa gỗ cầm tay cán lớn ak
3,váy đầm,đầm ren cổ tay ngắn cao cấp
4,dụng cụ đo kiểm tra khác,máy đo độ đồng tâm yan
...,...,...
12672,đèn vách,đèn vách xe đạp sgl
12673,hot deal ẩm thực,lẩu trà sữa dành người tại lẩu trà sữa
12674,sữa tắm,sữa tắm organic tảo xoắn pizu hãng phân phối c...
12675,bàn ghế trang điểm,bàn phấn mdf


In [4]:
# Tính số lượng dòng cho mỗi nhóm Label
label_counts = df['Label'].value_counts()
label_counts

bao đựng ốp lưng điện thoại     300
bàn ghế phòng khách             176
váy đầm                         158
bộ bàn ghế  phòng ăn bếp        115
đồ phong thủy                   107
                               ... 
xà đơn                            1
thiết bị viễn thông khác          1
máy đếm tần số                    1
sách điện tử  ebook readers       1
bánh pía                          1
Name: Label, Length: 1971, dtype: int64

In [18]:
# Lọc ra các nhãn có count < 50
labels_below_50 = label_counts[label_counts < 50].index.tolist()

print(len(labels_below_50))

1939


In [8]:
labels_below_50[:5]

['xe tải', 'đèn thả', 'gỗ mỹ nghệ', 'váy suông', 'các loại phụ kiện ô tô khác']

In [9]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

labels = labels_below_50
lbs = []
ns = []

for lb in tqdm(labels):
    label = lb
    label_encode = '%20'.join(lb.split())

    for p in range(1, 4):  # Duyệt qua 3 trang
        url = 'https://tiki.vn/search?q=' + label_encode + f'&page={p}'
        driver.get(url)

        # Loop through 53 elements to get the name
        for i in range(1, 53):
            try:
                element = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((
                        By.XPATH, 
                        f'/html/body/div[1]/div[1]/main/div/div[2]/div[1]/div[2]/div[1]/div[{i}]/div/div/div/a/span/div[2]/div[2]/div[2]/div[1]/h3 \
                        | /html/body/div[1]/div[1]/main/div/div[2]/div[1]/div[2]/div[1]/div[{i}]/div/div/a/span/div[2]/div[2]/div[2]/div[1]/h3'
                    ))
                )
                name = element.text.lower()

                if len(name) > 0:
                    ns.append(name)
                    lbs.append(label)
                
                if lbs.count(label) == 50:
                    break

            except Exception:
                # Nếu không tìm thấy phần tử, tiếp tục với trang tiếp theo
                break

        if lbs.count(label) == 50:
            break

# Close the driver
driver.quit()

100%|██████████| 1965/1965 [3:39:31<00:00,  6.70s/it]  


In [10]:
len(lbs), len(ns)

(96132, 96132)

In [13]:
df[df.Label =='đồ chơi  trang trí nội thất ô tô'][:5]

,Label,Name
57578,đồ chơi trang trí nội thất ô tô,"gối tựa đầu ô tô cao su non, kê cổ tựa lưng gh..."
57579,đồ chơi trang trí nội thất ô tô,miếng lót ghế tản nhiệt
57580,đồ chơi trang trí nội thất ô tô,ốp nội thất cross - ốp trang trí nội thất toyo...
57581,đồ chơi trang trí nội thất ô tô,thảm lót sàn xe ô tô hyundai elantra 2017- 202...
57582,đồ chơi trang trí nội thất ô tô,trang trí ô tô mở mắt và lắc đầu lợn đồ chơi b...


In [15]:
# Hàm để chuẩn hóa chuỗi
def normalize_label(label):
    # Loại bỏ khoảng trắng thừa và chuẩn hóa dấu cách
    return ' '.join(label.strip().split())

# Áp dụng hàm normalize_label cho từng phần tử trong cột 'Label'
df['Label'] = df['Label'].apply(normalize_label)

In [19]:
df[df.Label =='đồ chơi  trang trí nội thất ô tô'][:5]

,Label,Name


In [24]:
# Chuyển đổi danh sách thành DataFrame
df = pd.DataFrame({
    'Label': lbs,
    'Name': ns
})

# Xuất kết quả
print(df)

# Lưu DataFrame vào file CSV nếu cần
df.to_csv('output.csv', index=False, encoding='utf-8')

          Label                                               Name
0        mobile  điện thoại ai samsung galaxy s24 ultra 12gb/25...
1        mobile  điện thoại samsung galaxy a15 lte (8gb/128gb)-...
2        mobile  điện thoại samsung galaxy a05 (4gb/128gb) - đã...
3        mobile  điện thoại samsung galaxy a05s (4gb/128gb) - đ...
4        mobile                                    apple iphone 13
...         ...                                                ...
96127  bánh pía  bánh dừa nướng - đặc sản quảng nam 180g hiệu p...
96128  bánh pía  bánh gạo lứt mầm gufoods (bánh mầm) - giàu gab...
96129  bánh pía  set 5 gói bánh dừa nướng 180g - đặc sản quảng ...
96130  bánh pía  đặc sản sóc trăng - bánh pía sóc trăng đậu xan...
96131  bánh pía          bánh pía mỹ hiệp thành đặc biệt vàng 570g

[96132 rows x 2 columns]
